## Load Data

In [1]:
%qtconsole

In [2]:
import pandas as pd


In [3]:
from sklearn.model_selection import train_test_split

ImportError: DLL load failed: %1 is not a valid Win32 application.

In [ ]:
header = ['F' + str(i) for i in range(57)] + ['spam']

In [ ]:
spam = pd.read_csv("spambase.data", header = None, names = header)

In [ ]:
spam.head()

In [ ]:
spam.shape

### check any missing value

In [ ]:
spam.isnull().values.any()

### no missing values

## Simple EDA

### train/test split

In [ ]:
X = spam[['F' +str(i) for i in range(0, 57)]]
y = spam[['spam']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

### target distribution

In [ ]:
spam.spam.value_counts()